# ERA5 Data_Extraction



In [1]:
import numpy as np 
import pandas as pd 
import netCDF4
from datetime import datetime as Time
from datetime import timedelta as Delta
import h5py

def nc_to_pd(nc):
    try:
        Keys = list(nc.keys())
    except:
        Keys = list(nc.variables)
    Panda = {}
    for k in Keys:
        try:
            Panda[k]= np.array(nc[k] )
            PD = pd.DataFrame( Panda )
        except:
            Panda[k] = np.array(nc[k]).T[0]
            PD = pd.DataFrame( Panda )
            pass
    return PD , Panda

def number_to_date(number):
    start_date  = "01/01/1900"
    date_1 = pd.to_datetime(start_date)
    end_date = date_1 + pd.DateOffset(hours=number) 
    return end_date


def Dict_to_hdf(Dict , of ):
    try: HDF.close()   
    except: pass
    
    HDF = h5py.File(of, 'w' , driver='core')
    for k in Dict :    
        test = HDF.create_dataset( k , data = Dict[k]  )
    HDF.close()   


In [17]:


def ERA5_nc_hourly_to_daily( fp_netcdf , var = None ):
    nc = netCDF4.Dataset(fp_netcdf)
    Keys = list( nc.variables.keys() ) 
    lon = np.array(nc['longitude'])
    lat = np.array(nc['latitude'])
    time = np.array(nc['time'])
    if var == None : variable = Keys[-1]
    else:  variable = var
    date_0 = number_to_date(int(time[0]))
    date_f = number_to_date(int(time[-1]))

    start =  Time( date_0.year  , date_0.month , date_0.day  )
    end = Time( date_f.year  , date_f.month , date_f.day )

    delta = Delta(days = 1 )

    this_time =  start

    i = 0 
    out = {}
    Shape =  np.array(nc[variable]).shape
    print(Shape)
    
    while this_time <= end :
        j = i +24
        if len(Shape) == 4 : 
            img  = np.array(nc[variable][  i:j , 0 , : , :   ] )
        elif len(Shape) == 3 :
            img  = np.array(nc[variable][  i:j ,  :  ,  :   ] )
        T = np.mean( img , axis = 0) 
        out[str(this_time)[:10]] = T
        i = i +  24

        this_time = this_time + Delta(days = 1 )
        

    out['lat'] = lat 
    out['lon'] = lon 

    cache = [start , end , var]

    out['cache'] = [str(start) , str(end) , variable]
    
    return out , cache 


# Just one file 

In [2]:
fp_netcdf = r'd:\DatasetAUS\ERA5\New folder\2024.nc'
of  = r'd:\DatasetAUS\ERA5\New folder\era5-test.hdf5'
out , cache = ERA5_nc_hourly_to_daily( fp_netcdf   , 't2m' )



NameError: name 'ERA5_nc_hourly_to_daily' is not defined

In [3]:
fp = r'd:\DatasetAUS\ERA5\New folder\2024.nc'
nc = netCDF4.Dataset(fp)

In [13]:
't2m', 'cape', 'msl', 'tcw'

[datetime.datetime(2024, 1, 1, 0, 0),
 datetime.datetime(2024, 4, 6, 0, 0),
 't2m']

In [ ]:
Dict_to_hdf(out , of )

# Using a Folder of ERA5 hourly netcdf files

In [20]:


Folder_input = r"D:/DatasetAUS/ERA5/New folder/"
import os 

for k in ['cape', 'msl', 'tcw']:
    print(k)
    of_Folder = 'd:/DatasetAUS/ERA5/'+ k +'/' 
    try: os.mkdir( of_Folder)
    except: pass

    entries = os.listdir(Folder_input)

    for file in entries  :
        fp_netcdf  = Folder_input + file
        out , cache = ERA5_nc_hourly_to_daily( fp_netcdf   , k)
        of_name = str(cache[0])[:10] + '-' +str(cache[1])[:10] +'-' + cache[2] + '.hdf5'
        Dict_to_hdf(out , of_Folder  +of_name )



cape
(2306, 2, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
(8784, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
(8784, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
msl
(2306, 2, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
(8784, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
(8784, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
tcw
(2306, 2, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
(8784, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
(8760, 169, 281)
(8784, 169, 281)
(8760, 169, 281)
(8760, 169, 281)


# Joining HDF5 files

In [39]:
from numpy import dtype 
import os 


def JOIN_hdf5(FOLDER):
    # Should HAVE same lat , long 
    Dict ={}
    entries = os.listdir(Folder_input)
    for i in range(len(entries)):
        file  = entries[i] 
        fp  = Folder_input +  file 
        hdf = h5py.File(fp ) 
        Keys = list(hdf.keys())
        for k in Keys :
            Dict[k] = np.array(hdf[k] )
        hdf.close()

    return Dict




In [50]:
Folder_input = r"d:\DatasetAUS\ERA5\tp/"

hdf = JOIN_hdf5(FOLDER = Folder_input)


In [51]:
K = list(hdf.keys())

In [60]:
Shape = hdf[K[0]].shape
Image = np.zeros(Shape)[  :148 , 40 :220 ]
n = 0 
for k in K :
    if k not in ['lat' , 'lon' , 'cache']:
        Image += np.maximum(hdf[k] , 0 )[  :148 , 40 :220 ]
    n+=1
Image = Image/n

In [43]:
import tifffile as tff

In [61]:
tff.imwrite( r'D:\Projects\precipitation_AUS\results\Rasters/tp_subset_(8-45)S_(110-155)E.tif'  ,  Image*24*1000)

In [45]:
hdf['cache']

array([b'2024-01-01 00:00:00', b'2024-04-06 00:00:00', b'tcw'],
      dtype=object)

In [58]:
np.where(hdf['lat'] == -45)

(array([148], dtype=int64),)

In [22]:
hdf['lon']

array([100.  , 100.25, 100.5 , 100.75, 101.  , 101.25, 101.5 , 101.75,
       102.  , 102.25, 102.5 , 102.75, 103.  , 103.25, 103.5 , 103.75,
       104.  , 104.25, 104.5 , 104.75, 105.  , 105.25, 105.5 , 105.75,
       106.  , 106.25, 106.5 , 106.75, 107.  , 107.25, 107.5 , 107.75,
       108.  , 108.25, 108.5 , 108.75, 109.  , 109.25, 109.5 , 109.75,
       110.  , 110.25, 110.5 , 110.75, 111.  , 111.25, 111.5 , 111.75,
       112.  , 112.25, 112.5 , 112.75, 113.  , 113.25, 113.5 , 113.75,
       114.  , 114.25, 114.5 , 114.75, 115.  , 115.25, 115.5 , 115.75,
       116.  , 116.25, 116.5 , 116.75, 117.  , 117.25, 117.5 , 117.75,
       118.  , 118.25, 118.5 , 118.75, 119.  , 119.25, 119.5 , 119.75,
       120.  , 120.25, 120.5 , 120.75, 121.  , 121.25, 121.5 , 121.75,
       122.  , 122.25, 122.5 , 122.75, 123.  , 123.25, 123.5 , 123.75,
       124.  , 124.25, 124.5 , 124.75, 125.  , 125.25, 125.5 , 125.75,
       126.  , 126.25, 126.5 , 126.75, 127.  , 127.25, 127.5 , 127.75,
      

In [15]:
fp   = "D:\Projects\precipitation_AUS\models\CNN_RA_(3,3).hdf5" 

hd = h5py.File(fp)

In [16]:
K = list(hd.keys())

In [17]:
for k in K : print( k , np.array(  hd[k]))

Optimizer b'adam'
activation_function [b'relu' b'relu']
kernels [2 2]
layers [32 32]
n_epoch 15
name b'CNN_RA_(3,3)'
ord_keys [[b'CAL_P' b'PDIR' b'ERA5_tp' b'ERA5_cape' b'ERA5_t2m' b'ERA5_tcw']
 [b'CAL_P_ave_box(3, 3)' b'PDIR_ave_box(3, 3)' b'ERA5_tp_ave_box(3, 3)'
  b'ERA5_cape_ave_box(3, 3)' b'ERA5_t2m_ave_box(3, 3)'
  b'ERA5_tcw_ave_box(3, 3)']
 [b'CAL_P_std_box(3, 3)' b'PDIR_std_box(3, 3)' b'ERA5_tp_std_box(3, 3)'
  b'ERA5_cape_std_box(3, 3)' b'ERA5_t2m_std_box(3, 3)'
  b'ERA5_tcw_std_box(3, 3)']]
size_batch 500


In [2]:
import os

import tensorflow as tf
import keras

In [30]:

model = tf.keras.models.load_model(r"D:\Projects\precipitation_AUS\models\CNN_RA_(3,3).keras") 

model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_33 (Conv1D)              │ (None, 2, 32)          │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_34 (Conv1D)              │ (None, 1, 32)          │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,589 (29.65 KB)

 Trainable params: 2,529 (9.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,060 (19.77 KB)

In [26]:
import numpy as np

In [31]:
np.array(model.weights[0]).shape

(2, 6, 32)

In [42]:
a = np.random.random((1, 3 , 6))

In [43]:
a

array([[[0.94751309, 0.82940764, 0.23204183, 0.0575364 , 0.84346451,
         0.16636742],
        [0.15359446, 0.82367061, 0.94277446, 0.22923   , 0.28679829,
         0.96406169],
        [0.14822782, 0.99638536, 0.51691792, 0.2362563 , 0.29634398,
         0.13022798]]])

In [44]:
model.predict(a)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step


array([[3.2927382]], dtype=float32)

In [ ]:
model.b